In [39]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing import image_dataset_from_directory
from keras.applications import MobileNetV3Small
from keras.applications.mobilenet_v3 import preprocess_input


In [41]:
#Loading data

train_data = image_dataset_from_directory(
    directory="../data/SkinDisease/train",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(224,224),
    batch_size=32

)

test_data = image_dataset_from_directory(
    directory="../data/SkinDisease/test",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(224,224),
    batch_size=32,
    shuffle=False
)

Found 13898 files belonging to 22 classes.
Found 1546 files belonging to 22 classes.


In [42]:
# Preprocessing images
def preprocess(image, label):
    image = preprocess_input(image)

    return image, label

train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)


In [43]:
train_data = train_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

In [44]:
augment_data = Sequential([
    
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomFlip('vertical'),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomContrast(0.2),
    keras.layers.RandomTranslation(0.2, 0.2),
    keras.layers.RandomShear(x_factor=0.05, y_factor=0.05, fill_mode='constant')

])

In [45]:
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [46]:
base_model.trainable = False

In [47]:
print(len(base_model.layers))

157


In [48]:
# Building model
model = Sequential()

model.add(keras.layers.Input(shape=(224,224,3)))
model.add(augment_data)
model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(22, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

In [49]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 22)             │         2,838 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,015,814 (3.88 MB)

 Trainable params: 76,694 (299.59 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [50]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=['accuracy'])

In [51]:
history = model.fit(train_data, epochs=5, validation_data=test_data)

Epoch 1/5
435/435 ━━━━━━━━━━━━━━━━━━━━ 331s 738ms/step - accuracy: 0.1510 - loss: 2.9561 - val_accuracy: 0.2529 - val_loss: 2.6200
Epoch 2/5
435/435 ━━━━━━━━━━━━━━━━━━━━ 245s 564ms/step - accuracy: 0.2287 - loss: 2.6528 - val_accuracy: 0.2878 - val_loss: 2.4456
Epoch 3/5
435/435 ━━━━━━━━━━━━━━━━━━━━ 242s 557ms/step - accuracy: 0.2639 - loss: 2.5232 - val_accuracy: 0.3247 - val_loss: 2.3236
Epoch 4/5
435/435 ━━━━━━━━━━━━━━━━━━━━ 238s 546ms/step - accuracy: 0.2845 - loss: 2.4492 - val_accuracy: 0.3415 - val_loss: 2.2406
Epoch 5/5
435/435 ━━━━━━━━━━━━━━━━━━━━ 239s 549ms/step - accuracy: 0.2960 - loss: 2.3725 - val_accuracy: 0.3558 - val_loss: 2.1809


In [53]:
# Unfreezing some layers to train the model
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

In [54]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=['accuracy'])

In [55]:
history = model.fit(train_data, epochs=10, validation_data=test_data)

Epoch 1/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 350s 734ms/step - accuracy: 0.2381 - loss: 2.6701 - val_accuracy: 0.3318 - val_loss: 2.2724
Epoch 2/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 262s 603ms/step - accuracy: 0.2798 - loss: 2.4440 - val_accuracy: 0.3305 - val_loss: 2.2714
Epoch 3/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 268s 616ms/step - accuracy: 0.2881 - loss: 2.4058 - val_accuracy: 0.3370 - val_loss: 2.2452
Epoch 4/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 266s 611ms/step - accuracy: 0.3014 - loss: 2.3755 - val_accuracy: 0.3396 - val_loss: 2.2194
Epoch 5/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 254s 583ms/step - accuracy: 0.3008 - loss: 2.3599 - val_accuracy: 0.3480 - val_loss: 2.1982
Epoch 6/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 260s 597ms/step - accuracy: 0.3085 - loss: 2.3365 - val_accuracy: 0.3467 - val_loss: 2.1809
Epoch 7/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 292s 670ms/step - accuracy: 0.3135 - loss: 2.3151 - val_accuracy: 0.3532 - val_loss: 2.1619
Epoch 8/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 265s 608ms/step - accuracy: 0.3189 -